In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [2]:
data_dir = "tiny-imagenet-200/train"
transform = transforms.Compose([
    transforms.Resize((64, 64)),  
    transforms.ToTensor(),      
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder(root=data_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

print(f"Dataset size: {len(train_dataset)}")

Dataset size: 100000


In [5]:
class Small_AlexNet(nn.Module):
    def __init__(self, num_classes):
        super(Small_AlexNet, self).__init__()
        self.num_classes = num_classes

        self.net = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=5),
            nn.ReLU(),
            nn.BatchNorm2d(96),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(96, 192, 3, padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(192),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 192, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(in_features=(192*7*7), out_features=2048),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Linear(2048, out_features=self.num_classes)
        )

    def forward(self, x):
        x = self.net(x)
        x = x.view(-1, 192*7*7)
        x = self.classifier(x)
        return x

In [6]:
x = torch.randn(64, 3, 64, 64)
model = Small_AlexNet(200)
x = model(x)
x.shape

torch.Size([64, 200])

In [7]:
# hyperparameters
n_epochs = 30


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Current device for training:", device)

model = Small_AlexNet(num_classes=200).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

model.to(device)
all_loss = []

for epoch in range(n_epochs):
    model.train()
    
    for idx, (image, label) in enumerate(train_loader):
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
    
        all_loss.append(loss.item())
    
        if idx % 100 == 0:
            print(f"Epoch {epoch+1}/{n_epochs} - Step {idx}: Loss {loss.item():4f}")

Current device for training: cuda
Epoch 1/30 - Step 0: Loss 5.294644
Epoch 1/30 - Step 100: Loss 5.298115
Epoch 1/30 - Step 200: Loss 5.298069
Epoch 1/30 - Step 300: Loss 5.298064


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "alex_net/small_alex_net_v1.pth")

In [ ]:
data_dir = "tiny-imagenet-200/test"
transform = transforms.Compose([
    transforms.Resize((64, 64)),  
    transforms.ToTensor(),      
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset = ImageFolder(root=data_dir, transform=transform)

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for image, label in test_loader:
        image, label = image.to(device), label.to(device)
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

        correct += (predicted == label).sum().item()
        total += label.size(0)

accuracy = 100 * correct / total

In [ ]:
accuracy